In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
from IPython.display import Markdown
import re

from typing import Annotated, Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.document_loaders import PyMuPDFLoader, PyPDFLoader
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.documents.base import Document
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from pydantic import BaseModel, Field
from markitdown import MarkItDown
import dotenv
import os

dotenv.load_dotenv()

True

In [5]:
file_path = "./data/titanic.csv"
df = pd.read_csv(file_path)

llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

In [ ]:
def create_title_summary(df, file_path):
    text = f"""
            당신은 요약 전문가입니다.
           
            데이터프레임 : {df}

            데이터프레임의 정보를 보고 제목과 요약을 만들어냅니다.
            요약은 20글자 이내로 생성합니다.

            참고할 정보는 아래와 같습니다.

            파일 경로 : {file_path}

            1. 제목: 
            2. 요약: 

        """
    result = llm.invoke(text).content

    title = "Untitled"
    summary = "No Summary"    

    try:
        lines = result.split("\n")
        title = lines[0].replace("1. 제목: ", "").strip()
        summary = "\n".join(lines[1:]).replace("2. 요약:", "").strip()
    except:
        pass

    return title, summary

In [19]:
title, summary = create_title_summary(df, file_path)

여긴가


In [14]:
title

'타이타닉 승객 데이터'

In [15]:
summary

'생존자 및 승객 정보'